# Input

In [ ]:
import numpy as np
import MDAnalysis as mda

import matplotlib.pyplot as plt
%matplotlib inline

import re

import pandas as pd
import calvados as cal
from calvados import slab_conc

In [ ]:
records = cal.sequence.read_fasta('ects.fasta')
names = []
for name,rec in records.items():
    names.append(name)

names.remove('FUS') # comment out for positive control
# Alternatively, define names = [...] for subset of protein IDRs
print(names)
clean_names = names
    
temp = 293
aminoacids = "FYWHRKDEQNSTVILMAGPC"
residues = pd.read_csv('calvados/data/residues.csv').set_index('one')

fbase = 'slab'

plt.rcParams['font.size'] = 14
plt.rcParams['errorbar.capsize'] = 5

# Slab sims

In [ ]:
vv = pd.DataFrame(index=names,dtype=object)
ee = vv.copy()
dGs = {}
Lxys = {} # in Angstrom

## Lxys

In [ ]:
# xy dimension of slab

for name in names:
    path = f'{fbase}/{name}/{temp:d}'
    u = mda.Universe(f'{path}/top.pdb')
    if ('ECT2N' in name) and ('ECT2N_k' not in name):
        Lxys[name] = u.dimensions[0] / 10. # correct for some inconsistent early top.pdb
    else:
        Lxys[name] = u.dimensions[0]
    print(name, Lxys[name], u.dimensions)

## Timestraces

In [ ]:
plt.rcParams['font.size'] = 14

temp = 293

tmin = 2000
tmax = 7500

fig, ax = plt.subplots(1,len(names),figsize=(2*len(names),6),sharey=True)

for idx,name in enumerate(names):
    cmap = plt.cm.Blues
    clean_name = clean_names[idx]
       
    axij = ax[idx]
    ax[0].set(ylabel='Time [ns]')

    seq = records[name].seq
                       
    N = len(seq)
    mw = cal.sequence.calc_mw(seq) # Da
    hs = np.load(f'{fbase}/{name}/{temp:d}/{name}_{temp:d}.npy')
    
    L = Lxys[name] / 10. # in nm
    
    hconv_mM, hconv_mgml = cal.analysis.convert_h(hs,N,L,mw) # x mys
    
    axij.imshow(hconv_mgml[0:tmax,::10],cmap=cmap,vmin=0,vmax=600,
               aspect='auto')
    
    axij.set_title(clean_name,fontsize=14)#,rotation=90)

    if L in [17., 25.]:
        xticks = [50,150,250]
        xticklabels = [-100,0,100]
    else:
        xticks = [25,75,125]
        xticklabels = [-50,0,50]

    axij.set_xticks(xticks)
    axij.set_xticklabels(xticklabels)
    axij.set(xlabel=r'z [nm]')

fig.tight_layout()
# fig.savefig(f'{fbase}/slab_timetraces.pdf')

## Concentration profiles

In [ ]:
temp = 293

fig, ax = plt.subplots(figsize=(8,5))#,sharey=True)

xlim = (50,250)

for idx, name in enumerate(names):
    clean_name = clean_names[idx]
    color = None
    alpha = 1.

    print(clean_name)
    axij = ax
    L = Lxys[name] / 10. # nm
    seq = records[name].seq
    N = len(seq)

    mw = cal.sequence.calc_mw(seq) # Da
    # print(mw)
    # print(f'L={L}, n(AA)={N}')

    hs = np.load(f'{fbase}/{name}/{temp:d}/{name}_{temp:d}.npy')

    hconv_mM, hconv_mgml = cal.analysis.convert_h(hs,N,L,mw)

    # print(hs.shape)
    # print(f'total protein conc [mM]: {np.mean(hconv_mM):.2f}')
    # print(f'total AA conc [mM]: {np.mean(hconv_mM)*N:.2f}')

    xs = np.arange(0,hs[0].shape[0]/10.,0.1)

    ls = 'solid'

    axij.plot(xs,np.mean(hconv_mgml[tmin:tmax],axis=0),
              label=clean_name,lw=1.0,c=color,ls=ls,alpha=alpha)

    pden = 0.5

    if name == 'Gb_DFC2':
        pdil = 4
    elif name == 'Gb_DFF1':
        pdil = 4
    else:
        pdil = 6
    vv, ee = cal.slab_conc.calcProfile(seq,name,temp,L,vv,ee,fbase=fbase,tmin=tmin,tmax=tmax,plot=False,pdil=pdil,pden=pden)

print('---------')
print('csat')
print(vv)
print('---------')
print('errors')
print(ee)

axij = ax    
axij.set(xlim=xlim)
axij.set(yscale='log')
axij.set(xlabel='z [nm]',ylabel='Density [mg/mL]')
axij.legend(loc=1,fontsize=10,ncol=2)
fig.tight_layout()

## csat

In [ ]:
T = 293.

fig, ax = plt.subplots(figsize=(12,4))#,sharey=True)

for idx,name in enumerate(names):
    print(name)
    
    seq = records[name].seq
    N = len(seq)
    
    c_dil = vv.loc[name][f'{temp}_dil']   
    e_dil = ee.loc[name][f'{temp}_dil']
    
    if name == 'ECT2N':
        ax.axhline(c_dil,c='gray',ls='dotted')

    alpha = 1.
    ax.errorbar(idx,c_dil,yerr=e_dil,marker='o',capsize=5,alpha=alpha)#,c=colors[idx])
    ax.set_xticks(np.arange(len(clean_names)))
    ax.set_xticklabels(clean_names,rotation=90)
ax.set_yscale('log')
ax.set(ylabel=r'c$_{sat}$ [mM]')
ax.set(ylim=(6e-3,3))

ax.grid(ls='dotted',alpha=0.4)

fig.tight_layout()
# fig.savefig('figures/csat_all.pdf')

## dG

In [ ]:
T = 293.

fig, ax = plt.subplots(figsize=(12,4))#,sharey=True)

for idx,name in enumerate(names):
    seq = records[name].seq
    
    N = len(seq)
    
    c_dil = vv.loc[name][f'{temp}_dil']
    c_den = vv.loc[name][f'{temp}_den']
    
    e_dil = ee.loc[name][f'{temp}_dil']
    e_den = ee.loc[name][f'{temp}_den']
    
    dG, dG_error = cal.analysis.calc_dG(c_dil,e_dil,c_den,e_den)
    dGs[name] = dG
    
    if name == 'ECT2N':
        ax.axhline(dG,c='gray',ls='dotted')
    
    ax.errorbar(idx,dG,yerr=dG_error,marker='o',capsize=5,alpha=alpha)
    ax.set_xticks(np.arange(len(vv[f'{temp}_dil'])))
    ax.grid(ls='dotted',alpha=0.4)
    ax.set_xticklabels(clean_names,rotation=90)

ax.set(ylabel='$\Delta G_T$ [k$_B$T]')

fig.tight_layout()
# fig.savefig('figures/dGs_all.pdf')

# Sequence features

## AA distribution

In [ ]:
fig, ax = plt.subplots(len(names),1,figsize=(8,24),sharex=True,sharey=True)

ct = 0
for idx, name in enumerate(names):
    print(ct,name)
    if name in ['FUS','A1','MRB1']:
        # ct += 1
        continue
    clean_name = name
    # clean_name = clean_names_ects[idx]
    f = "".join(records[name].seq)
    # print(f[del2[0]:del2[1]])
    axij = ax[ct]
    aa_dist = np.zeros((len(aminoacids)))
    for aa in f: # convert 1-based to 0-based
        res = re.search(aa,aminoacids).start()
        if res != None:
            aa_dist[res] += 1
    aa_dist /= np.sum(aa_dist)
    
    axij.bar(np.arange(len(aminoacids)),aa_dist)#,width)

    axij.set(xlim=(-0.5,len(aminoacids)-0.5),ylim=(0,0.18))
    axij.set(ylabel='$f$')
    axij.text(-0.3,0.12,clean_name)
    for i in range(len(aminoacids)):
        axij.axvline(i,color='gray',ls='dashed',lw=0.5,alpha=0.3)
    ranges = [(0,2),(4,5),(8,11),(17,19)]
    width=0.5
    for r in ranges:
        axij.axvspan(r[0]-width,r[1]+width,color='gray',alpha=0.2)
    axij.set_yticks(np.arange(0,0.15,0.1))
    axij.axhline(0.1,color='gray',ls='dashed',lw=0.5,alpha=0.3)
    ct += 1
    # axij.axvspan(-0.4,2.4,color='gray',alpha=0.3)
ax[-1].set(xticks=np.arange(len(aminoacids)))
ax[-1].set(xticklabels=aminoacids)
fig.tight_layout(h_pad=0.)

## Charges along seq

In [ ]:
fig, ax = plt.subplots(13,3,figsize=(12,18))#,sharey=True)

for idx,name in enumerate(names):
    prot = records[name]
    seq = prot.seq
    print(name, len(seq))
    qs, qs_abs = cal.sequence.get_qs(seq)
    
    i = idx // 3
    j = idx % 3
    
    axij = ax[i,j]
    xs = np.arange(1,len(qs)+1)
    axij.plot(xs,np.cumsum(qs),lw=1.5)
    xmax = len(qs)+1
    xmin = 1
    axij.set(xlim=(np.min(xs),np.max(xs)))
    if len(qs) > 200:
        step = 100
    else:
        step = 50
    axij.set_xticks(np.arange(step,np.max(xs)+1,step))

    ax[i,0].set(ylabel=r'$\Sigma(q)$')
    axij.set_title(clean_names[idx],fontsize=14)
    axij.axhline(0,c='gray')
for jdx in range(3):
    ax[-1,jdx].set(xlabel='Residue')
fig.tight_layout()
# fig.savefig(f'slab/plant_idrs/q_along_seq.pdf')

## Lambda vs dG (2D plot)

In [ ]:
cladedict = {
    'A' : ['Aa_DFA2', 'Aa_DFA1', 'Nc_DFA', 'Atr_DFA', 'Gb_DFA2', 'Gb_DFA1'],
    'B' : ['Gb_DFB', 'Atr_DFB', 'Nc_DFB', 'Aa_DFB', 'Cri_DFAB'],
    'C' : ['Aa_DFC3', 'Atr_DFC2', 'Nc_DFC2', 'Aa_DFC2', 'Aa_DFC1', 'Nc_DFC1', 'Atr_DFC1', 'Gb_DFC1', 'Gb_DFC2'],
    'D' : ['Cri_DFD2', 'Cri_DFD1', 'Gb_DFD', 'Atr_DFD', 'Nc_DFD', 'Aa_DFD'],
    'E' : ['Mpo_YTHDF', 'Cp_YTHDF', 'Pp_YTHDF1', 'Pp_YTHDF2', 'Sm_YTHDF',
          'Ita_DFE1', 'Ita_DFE2', 'Ita_DFE3'],
    'F' : ['Gb_DFF1', 'Gb_DFF2', 'Cri_DFF3', 'Cri_DFF2', 'Cri_DFF1']
}

clade_to_color = {
    'A' : 0,
    'B' : 1,
    'C' : 2,
    'D' : 3,
    'E' : 4,
    'F' : 5,
}

df = pd.read_csv('Dicot_IDRs.csv',sep=';',header=0)
df = df[~df['Name_FloTel'].str.contains('Atr')]
df = df.set_index('Name_FloTel')

name_to_color = {}
name_to_clade = {}

colors = [
    [154/255,153/255,41/255],
    [102/255,110/255,36/255],
    [167/255,31/255,148/255],
    [225/255,0,20/255],
    [127/255,54/255,48/255],
    [255/255,121/255,0]
]

In [ ]:
for name in names:
    clade = None
    if name[:3] == 'ECT':
        name_in_df = name[:-1]
    else:
        name_in_df = name

    if name_in_df in df.index:
        clade = df.loc[name_in_df,'CLADE'][0]
        color = colors[clade_to_color[clade]]
    else:
        for cl, clitem in cladedict.items():
            if name_in_df in clitem:
                clade = cl
                color = colors[clade_to_color[clade]]
                break
    if clade == None:
        color = 'black'

    name_to_clade[name] = clade
    name_to_color[name] = color

In [ ]:
fig, ax = plt.subplots(figsize=(6,5))
clade_label = [1 for _ in range(6)]

for idx, name in enumerate(names):
    color = name_to_color[name]
    clade = name_to_clade[name]
    if clade != None:
        clidx = clade_to_color[clade]
    if name in ['A1','FUS']:
        continue
    seq = records[name].seq
    N = len(seq)

    lambdas_mean = cal.sequence.mean_lambda(seq,residues)

    offset = 0.002
    
    for jdx,feature in enumerate([lambdas_mean]):
        axij = ax
        if np.isnan(dGs[name]): # unconverged --> 0
            dG = 0.
            marker = 'x'
            continue
        else:
            dG = dGs[name]
            marker = 'o'
        if clade == None:
            axij.plot(dG,feature,marker,color=color)
        else:
            if clade_label[clidx] == 1:
                axij.plot(dG,feature,marker,color=color,label=f'DF-{clade}')
                clade_label[clidx] = 0
            else:
                axij.plot(dG,feature,marker,color=color)
        axij.text(dG,feature,clean_names[idx],fontsize=9,color=color)
        
for jdx, lab in enumerate([r'$\overline{\lambda}$']):
    axij = ax
    axij.set(xlabel=r'$\Delta G$ [k$_B$T]')
    axij.set(ylabel=lab)
    axij.grid(True,alpha=0.3)

ax.legend()
ax.set(xticks=(np.arange(-6,0.1,2)))
fig.tight_layout()
# fig.savefig('figures/simulation_2Dplots.pdf')